In [2]:
import optimiserBank as opt
import functionBank as func
import matplotlib.pyplot as plt
import importlib
importlib.reload(opt)
importlib.reload(func)
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import qmc 
from itertools import product
from pymoo.problems import get_problem

dtlzProblems = ['dtlz1','dtlz2']

n_var = 4
n_obj = 2

scalarisingList = [
    func.HypI,
    func.chebyshev,
    # func.weightedSum,
    func.EWC,
    func.weightedPower,
    # func.weightedNorm,
    # func.augmentedChebychev,
    # func.modifiedChebychev,
    func.PBI,
    # func.PAPBI,
]

weights = np.array((0.5, 0.5))


for function in dtlzProblems:

    print('current function  = ', function)

    problem, bounds = func.getPyMooProblem(function, n_var, n_obj)

    initSampleSize = 50
    # bounds = np.array(value)
    lowBounds = bounds[:, 0]
    highBounds = bounds[:, 1]

    # Generate one Latin Hypercube Sample (LHS) for each test function,
    # to be used for all optimisers/scalarisers using a population size of 20
    sampler = qmc.LatinHypercube(
        d=bounds.shape[0]
    )  # Dimension is determined from bounds
    sample = sampler.random(n=initSampleSize)
    initPopulation = qmc.scale(sample, lowBounds, highBounds)

    # Check for and systematically replace NaN values in initial population
    # Requires evaluating initial population
    initialObjvTargets = np.empty((0, n_obj))


    for i in range(initSampleSize):

        newObjvTgt = opt.MOobjective_function(initPopulation[i], problem, n_obj)
        initialObjvTargets = np.vstack((initialObjvTargets, newObjvTgt))

    print("Initial Population:")
    print(initPopulation)
    print("initial targets:\n", initialObjvTargets )

    for scalarisingFunction in scalarisingList:

        print(scalarisingFunction.__name__)


        bayesianRun = opt.bayesianOptimiser(
            bounds,
            initSampleSize,
            problem,
            scalarisingFunction,
            n_obj,
            weights,
            useInitialPopulation=True,
            initialPopulation=initPopulation,
            initialObjvValues=initialObjvTargets,
            maxFE=100
        )
        bayesianRun.runOptimiser()

        features = np.loadtxt("BOMinMaxFeatures.txt")
        np.savetxt(
            f"BOFeatures{function}{scalarisingFunction.__name__}.txt", features
        )

        scalarisedTargets = np.loadtxt("BOMinMaxScalarisedTargets.txt")
        np.savetxt(
            f"BOScalarisedTargets{function}{scalarisingFunction.__name__}.txt",
            scalarisedTargets,
        )

        objtTargets = np.loadtxt("BOMinMaxObjectiveTargets.txt")
        np.savetxt(
            f"BOObjtvTargets{function}{scalarisingFunction.__name__}.txt",
            objtTargets,
        )



current function  =  dtlz1
Initial Population:
[[0.76792524 0.54288491 0.69339668 0.08709038]
 [0.15555391 0.57039066 0.62647047 0.49286556]
 [0.45905151 0.83634726 0.61361504 0.15155113]
 [0.93981826 0.3011826  0.8768445  0.1370718 ]
 [0.68373028 0.05376856 0.38811197 0.75436656]
 [0.84411362 0.72112868 0.20119256 0.38120204]
 [0.59979188 0.97738572 0.55474076 0.97512957]
 [0.54028626 0.46222463 0.79512978 0.8757026 ]
 [0.86827326 0.69588374 0.50692324 0.35185452]
 [0.2026051  0.77095303 0.76205228 0.650074  ]
 [0.30759662 0.29756841 0.32588024 0.03797515]
 [0.37889393 0.87305677 0.81111542 0.42836734]
 [0.89604912 0.93882975 0.06716344 0.58926975]
 [0.4966518  0.64409276 0.13527671 0.66234018]
 [0.71887907 0.79514632 0.09655584 0.23271387]
 [0.467877   0.89069842 0.25187521 0.3120126 ]
 [0.43656084 0.75555861 0.94141469 0.16681599]
 [0.56004321 0.36086075 0.41892115 0.44362787]
 [0.33585169 0.21479911 0.58804988 0.26739392]
 [0.97655884 0.13024403 0.31598507 0.77779883]
 [0.28635924 

KeyboardInterrupt: 